# Introduction

This analysis to to generate prediction profiles and importance scores for given sv40 sequence from trained model.


# Computational setup

## Environment

In [1]:
print("Launching packages...")
import pandas as pd 
import os
import json
import numpy as np
import comet_ml
from comet_ml import Experiment
from collections import OrderedDict

import os
import sys
import pandas as pd
import numpy as np
from pybedtools import BedTool
from tqdm import tqdm
from bpnet.cli.contrib import bpnet_contrib
from bpnet.cli.modisco import cwm_scan
from bpnet.cli.contrib import ContribFile
from bpnet.BPNet import BPNetSeqModel
from concise.preprocessing import encodeDNA
from bpnet.simulate import random_seq, insert_motif
from bpnet.plot.tracks import plot_tracks, to_neg

from pathlib import Path
from concise.preprocessing import encodeDNA
from concise.preprocessing.sequence import one_hot2string, DNA
from bpnet.BPNet import BPNetSeqModel
import warnings
warnings.filterwarnings("ignore")


# Settings
os.chdir(f'/n/projects/kd2200/publication/bpnet/analysis/')

# Pre-existing variables
fasta_file = f'/n/projects/kd2200/publication/bpnet/fasta/mm10.fa'
mdir = f'/n/projects/kd2200/publication/bpnet/model/dataspec.yaml_default_fold_5/'
can_input=pd.read_csv("/n/projects/kd2200/publication/bpnet/analysis/csv/sv40/candidate_sv40.csv")

Launching packages...


Using TensorFlow backend.


2023-12-14 16:29:27,396 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

/home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/bpnet-0.0.21-py3.6.egg/bpnet/plot/heatmaps.py:6: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid1.colorbar module was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use matplotlib.colorbar instead.
  from mpl_toolkits.axes_grid1.colorbar import colorbar
2023-12-14 16:29:30,511 [INFO] Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-12-14 16:29:30,513 [INFO] Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-12-14 16:29:30,514 [INFO] NumExpr defaulting to 8 threads.


In [2]:
# load model
model = BPNetSeqModel.from_mdir(mdir)

2023-12-14 16:29:37,789 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



2023-12-14 16:29:37,824 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



2023-12-14 16:29:38,623 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



2023-12-14 16:29:38,625 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



2023-12-14 16:29:38,627 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



...

2023-12-14 16:29:39,058 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/tensorflow_core/python/ops/distributions/multinomial.py:217: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
Use `tf.cast` instead.


2023-12-14 16:29:39,063 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/bpnet-0.0.21-py3.6.egg/bpnet/losses.py:30: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


2023-12-14 16:29:40,150 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



2023-12-14 16:29:40,516 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



# generate random sequences for sv40 (100bp) region to be embeded in it

In [3]:
def generate_random_seq(seqlen, weights = [.25, .25, .25, .25]):
    """
    Purpose: Generate a random DNA sequence of a specified length.
    """
    import random
    seq = random.choices(['A','C','G','T'], weights = weights, k=seqlen)
    return(''.join(seq))

import random
random.seed(10)
trials = 64
random_seqs = [generate_random_seq(seqlen = 900) for i in range(trials)]
#random_seqs (took tthese sequences into excel sheet and added sv40 sequence at 500 bp position) (offcourse using a formula!)

read data


In [4]:
print("Reading sequences...")
can_input.head(n=5)

Reading sequences...


,name,sequence
0,1,GCGATTGAGCATTATGCCGCGGATTTGAAATCCTCGCAGTAACACT...
1,2,GTGTCGGAGTAAGGGTCGGTCTTGTGCTCCGGTCTCAGGCGCAACG...
2,3,CATCTACAGTCTCGAATTAACTCTGCATGTAAGCTACGTGGACAAT...
3,4,ATCAGTCTAACTGGGGATTCTCGACGTTTGAGATTCGTCGCGTCCC...
4,5,GTGGGTATCCGTTGCTTCCTTGCCTTCCGATCGAATCCTCACCCGT...


Apply the trained tsc model to predict and get importance scores.

1. 1-hot encode the data.  
2. Predict the ChIP-nexus profiles using BPNet 

In [5]:
#1-hot-encode the data
can_seq = encodeDNA(can_input.sequence.str.upper())
#Predict the data
can_seq_preds = model.predict_all(can_seq, contrib_method='deeplift')

2023-12-14 16:29:42,810 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/bpnet-0.0.21-py3.6.egg/bpnet/heads.py:350: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Instructions for updating:
dim is deprecated, use axis instead


2023-12-14 16:29:42,814 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/bpnet-0.0.21-py3.6.egg/bpnet/heads.py:318: calling softmax (from tensorflow.python.ops.nn_ops) with dim is deprecated and will be removed in a future version.
Instructions for updating:
dim is deprecated, use axis instead


2023-12-14 16:29:47,098 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2023-12-14 16:29:47,100 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/Keras-2.2.4-py3.6.egg/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2023-12-14 16:29:52,227 [WARNING] From /home/kd2200/anaconda3/envs/bpnet/lib/python3.6/site-packages/deepexplain/tensorflow/methods.py:287: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running "deeplift" explanation method (5)
Model with multiple inputs:  True
DeepExplain: running 

## Reformat predictions 

In [6]:
def bpnet_format_dict_to_df(dict_obj):
    tead4_prediction = pd.DataFrame(data= dict_obj['p']["tead4"], columns = ["pos", "neg"])
    tfap2c_prediction = pd.DataFrame(data= dict_obj['p']["tfap2c"], columns = ["pos", "neg"])
    cdx2_prediction = pd.DataFrame(data= dict_obj['p']["cdx2"], columns = ["pos", "neg"])
    gata3_prediction = pd.DataFrame(data= dict_obj['p']["gata3"], columns = ["pos", "neg"])
    yap1_prediction = pd.DataFrame(data= dict_obj['p']["yap1"], columns = ["pos", "neg"])   
    
    tead4_importance = pd.DataFrame(data= dict_obj['i']["tead4"], columns = ["score"])
    tfap2c_importance = pd.DataFrame(data= dict_obj['i']["tfap2c"], columns = ["score"])
    cdx2_importance = pd.DataFrame(data= dict_obj['i']["cdx2"], columns = ["score"])
    gata3_importance = pd.DataFrame(data= dict_obj['i']["gata3"], columns = ["score"])
    yap1_importance = pd.DataFrame(data= dict_obj['i']["yap1"], columns = ["score"])

    tead4_data = pd.concat([tead4_prediction, tead4_importance], axis=1)
    tfap2c_data = pd.concat([tfap2c_prediction, tfap2c_importance], axis=1)
    cdx2_data = pd.concat([cdx2_prediction, cdx2_importance], axis=1)
    gata3_data = pd.concat([gata3_prediction, gata3_importance], axis=1)
    yap1_data = pd.concat([yap1_prediction, yap1_importance], axis=1)

    tead4_data["factor"]='Tead4'
    tfap2c_data["factor"]='Tfap2c'
    cdx2_data["factor"]='Cdx2'
    gata3_data["factor"]='Gata3'
    yap1_data["factor"]='Yap1'

    tead4_data['position']=range(tead4_data.shape[0])
    tfap2c_data['position']=range(tfap2c_data.shape[0])
    cdx2_data['position']=range(cdx2_data.shape[0])
    gata3_data['position']=range(gata3_data.shape[0])
    yap1_data['position']=range(yap1_data.shape[0])


    output_data = pd.concat([tead4_data, yap1_data,tfap2c_data, cdx2_data,gata3_data], axis=0) 
    return(output_data)

In [7]:
def bpnet_get_dict_output(obj, tasks):
    out = dict(p=obj['preds'],
               i={t: obj['seq'] * (obj['contrib_score'][f'{t}/profile']['pos'] + obj['contrib_score'][f'{t}/profile']['neg'])/2 
                  for ti,t in enumerate(tasks)})    
    for i in range(len(out['i'])):
        out['i'][bpnet.tasks[i]] = np.sum(out['i'][bpnet.tasks[i]], axis=1).tolist() #add quick loop to format seqs
        
    output_df=bpnet_format_dict_to_df(out)
    
    return(output_df)

# Export predictions

In [8]:
def export_predictions_all(preds_all, tasks, contrib_type = 'profile'):
    import sys
    sys.path.insert(0, f'/n/projects/kd2200/publication/bpnet/analysis/scripts/bpnet/scripts')
    from data_format_functions import tidy_bpnet_predictions_nexus 
    
    #Create empty pd.df to write into 
    preds_df = pd.DataFrame()
    contrib_df = pd.DataFrame() 
    
    #Tidy perturbations into a pd.df
    for task in tasks:
        for r in range(len(preds_all)):
            p = {task: preds_all[r]['pred'][task]}
            p_df = tidy_bpnet_predictions_nexus(predictions = p, tasks = [task])
            p_df['task'] = task
            p_df['region_idx'] = r
            preds_df = preds_df.append(p_df) 
            c_df = pd.DataFrame(preds_all[r]['contrib_score'][f'{task}/{contrib_type}']*preds_all[r]['seq'], columns = ['A','C','G','T'])
            c_df['task'] = task
            c_df['position'] = list(range(c_df.shape[0]))
            c_df['region_idx'] = r
            contrib_df = contrib_df.append(c_df)
    return(preds_df, contrib_df)
    
can_predictions, can_contribution = export_predictions_all(preds_all=can_seq_preds, tasks=['tead4', 'yap1','cdx2', 'gata3','tfap2c'])


In [9]:
can_contribution

,A,C,G,T,task,position,region_idx
0,-0.000000,-0.000000,0.000062,-0.000000,tead4,0,0
1,-0.000000,0.000292,-0.000000,-0.000000,tead4,1,0
2,-0.000000,0.000000,-0.000213,0.000000,tead4,2,0
3,-0.000004,0.000000,0.000000,-0.000000,tead4,3,0
4,-0.000000,0.000000,0.000000,-0.000459,tead4,4,0
...,...,...,...,...,...,...,...
995,-0.002354,-0.000000,0.000000,-0.000000,tfap2c,995,63
996,-0.000000,-0.000000,-0.000000,0.000902,tfap2c,996,63
997,-0.000000,-0.000000,0.002916,-0.000000,tfap2c,997,63
998,-0.001177,-0.000000,-0.000000,0.000000,tfap2c,998,63


In [10]:
can_predictions

,prediction,position,task,strand,region_idx
0,0.006674,0,tead4,+,0
1,0.005848,1,tead4,+,0
2,0.004305,2,tead4,+,0
3,0.005882,3,tead4,+,0
4,0.003217,4,tead4,+,0
...,...,...,...,...,...
995,-0.012054,995,tfap2c,-,63
996,-0.017931,996,tfap2c,-,63
997,-0.009926,997,tfap2c,-,63
998,-0.008732,998,tfap2c,-,63


In [30]:
#Write to csv for import to R
can_predictions.to_csv("csv/sv40/predictions_sv40_full.csv",index = False)
can_contribution.to_csv("csv/sv40/contributions_sv40_full.csv",index = False)